# Registery Query MVP Demo Notebook 
> A tutorial of RRAP Registry Query using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, provenance]
<!-- - image: images/chart-preview.png -->

## About

This notebook is a demonstration of some of capabilities of RRAP Provenance with notebooks.

With `RRAP-IS Provenance` system can be queried to determine all 'Activities', 'Entities' and 'Actors' associated with particular 'Outputs', 'Inputs', or 'Reef Interventions'

Please read documenation on **Provenance** to understand terms used in this notebook and the future systems 

### Import general dependencies

In [25]:
import requests
import os
import sys
import json
from IPython.core.display import display, HTML
from IPython.display import IFrame
sys.path.append("python_packages/rrap_packages")
    
from python_packages.rrap_packages.token_manager import DeviceFlowManager

### Define global variables, base urls
Note the use structure of the base url, pointing to test servers.  If you wish to test your code first, please use the test servers (listed below).  Note that these may not retain data and could be down at the time of testing.

Remove `testing` from the url to use the production urls.  Please be careful using production urls, filling the system with test data will require considerable effort to cleanup.

In [26]:

data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display('Checking base urls')
# [display(Testing':',value) for key, value in base_urls.items()]
for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

In [ ]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

### Authenticate

In [ ]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/items/list-all-datasets"
endpoint = data_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

## Demonstrations
[Use cases](https://confluence.csiro.au/display/RRAPIS/Provenance+use+cases)


## List all registered models

In [31]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/list"
endpoint = registry_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

Token validation failed due to error: Signature has expired.
Refreshing using refresh token

{
    "complete_item_count": 5,
    "items": [
        {
            "created_timestamp": 1658380814,
            "display_name": "CoCoNet-model (commit 8d6a1b7)",
            "id": "10378.1/1687602",
            "item_category": "ENTITY",
            "item_subtype": "MODEL",
            "name": "CoCoNet-model-commit-8d6a1b7",
            "updated_timestamp": 1658380814,
            "url": "https://github.com/gbrrestoration/CoCoNet-model/commit/8d6a1b7"
        },
        {
            "created_timestamp": 1658219443,
            "display_name": "Jonno's Awesome Model",
            "id": "10378.1/1687582",
            "item_category": "ENTITY",
            "item_subtype": "MODEL",
            "name": "JonnoModel1.0",
            "updated_timestamp": 1658219443,
            "url": "https://www.fake.model.com"
        },
        {
            "created_timestamp": 1658380827,
            "display_

## Fetch registered model
From the above output copy the handle id and use it to fetch 

In [33]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/fetch"
params = "?id=10378.1/1687582"
endpoint = registry_api + postfix + params

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "item": {
        "created_timestamp": 1658219443,
        "display_name": "Jonno's Awesome Model",
        "id": "10378.1/1687582",
        "item_category": "ENTITY",
        "item_subtype": "MODEL",
        "name": "JonnoModel1.0",
        "updated_timestamp": 1658219443,
        "url": "https://www.fake.model.com"
    },
    "item_is_seed": false,
    "status": {
        "details": "Successfully retrieved complete item and parsed into current data model.",
        "success": true
    }
}
